In [ ]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time
import _init_paths

In [ ]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=sequence[index-1],
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [ ]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [ ]:
def evaluate(results):
    
    hist_cumul = 0*results[0]
    for hist in results:
        hist_cumul+=hist
    result = {}
    

    acc = np.diag(hist_cumul).sum() / hist_cumul.sum()
    # per-class accuracy
    pc_acc = np.diag(hist_cumul) / hist_cumul.sum(1)
    mean_accuracy= np.nanmean(pc_acc)
    # per-class IU
    iu = np.diag(hist_cumul) / (hist_cumul.sum(1) + hist_cumul.sum(0) - np.diag(hist_cumul))
    mean_iu= np.nanmean(iu)
    freq = hist_cumul.sum(1) / hist_cumul.sum()
    freq_weighted_iu=(freq[freq > 0] * iu[freq > 0]).sum()
    
    print("Accuracy :",acc)
    print("per-class accuracy :",mean_accuracy)
    print("per-class IU :",mean_iu)
    print("per-class IU (frequency weighted) :",freq_weighted_iu)
    
    return acc

In [ ]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [ ]:
ICIPpath = '/home/jogue/workspace/ICIPmodels/'
weights_path_fuse='/home/jogue/workspace/ICIPmodels/fuse_weights.caffemodel'
fuse_solver = ICIPpath + 'solver.prototxt'
max_iters = 7 #95

In [ ]:
solver = caffe.SGDSolver(fuse_solver)

In [ ]:
if weights_path_fuse is not None:
    print('Loading pretrained model weights from :',weights_path_fuse)
    solver.net.copy_from(weights_path_fuse)
    print('done')
    print('')

In [ ]:
for idx in log_progress(range(max_iters), title='iteration', removeAtTheEnd=False):
    #while solver.iter < max_iters:
    #solver.step(1)
    fuseNet.backward()
    fuseNet.forward()
    for layer in fuseNet.layers:
        for blob in layer.blobs:
            blob.data[...] -= blob.diff